# Arbitrage auto detection using matrices
 
 https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1096549
 
 [original paper method](https://d1wqtxts1xzle7.cloudfront.net/34802958/06F167EF-B243-48ED-8C45-F7466B3136EB-WebPublishings-How_to_make_decision_AHP.pdf?1411208782=&response-content-disposition=inline%3B+filename%3DHow_to_make_a_decision_The_Analytic_Hier.pdf&Expires=1616709686&Signature=TPWpLboiXquvLmKWLCmdRi74uUtv7J4vrR0qXa4uFRjQpoEuZ0YBYZdnmg8xtTy8IJJjkaTV6MK52iNaUwEPn~qi67Sw1J2zx38o8zlAkwmNqJ2MZA5vn75Tk3NEm2jCpcCrjyn~UgUYXyZsKdf8007fAxrLIRZ~vS9Lj91tvZh2QUYpzp9CKqu9osR9lkCWu0SreKRYEnu~XTS0gvocjbt1gzYtQ7wpFG-QDih-7gNszyreN1G5IJBUg4x5L8dUBSXEA48UYoWn84gLoOcnDpY7PYPpuvTSNCr~HdymMH3MRJvuJtpoE10Qx058pdjHzPd2hhqLW8JxE8TBLKLC-w__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA)

In [70]:
import urllib
import urllib.request
import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [523]:
def get_symbol_prices(symbol):
    binance_uri = binance_url + binance_endpoints['order_book'][1] + f'?symbol={symbol}&limit=5'

    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))

    prices = [float(price) for price, quantity in JSON_object['bids']]
    weights = [float(quantity) for price, quantity in JSON_object['bids']]
    buy = np.average(prices, weights=weights)

    prices = [float(price) for price, quantity in JSON_object['asks']]
    weights = [float(quantity) for price, quantity in JSON_object['asks']]
    sell = np.average(prices, weights=weights)

    return buy, sell

In [624]:
def calculate_arbitrage(vecmax):
    B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
    C = np.divide(A, B)

    C_max = np.unravel_index(C.argmax(), C.shape)
    C_min = np.unravel_index(C.argmin(), C.shape)

    # Algorithm
    # BUY = division
    # SELL = multiply
    print('Arbitrage orders: ', end="")
    if C_max[0]==C_min[1] and C_max[1]==C_min[0]:
        # Direct arbitrage
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # and sell it for currency C_max[0] in location C_max[1]
        print('DIRECT')
        aer = 1/np.real(C[C_min]*C[C_max])-1

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]:6}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[0]]}({A[C_max[0],C_min[0]]}) in {locations[C_max[1]]:6}')

        operation = 1/(A[C_min]*A[C_max])
        print(f'1/{A[C_min]:.4}*{A[C_max]:.4} = {operation:.5}')

    elif C_max[0]==C_min[0] or C_max[1]==C_min[1]:
        # Triangular arbitrage
        if C_max[0]==C_min[0]:
            # Arbitrage elements in the same row
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[1] in location C_max[1]
            # then buy currency C_max[1] in location C_max[1]
            print('TRIANGULAR ROW')
            aer = np.real(1/C[C_min]*C[C_min[0],C_max[1]]/C[C_min[1],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
            print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[1]]}({A[C_min[0],C_max[1]]}) in {locations[C_max[1]]}')
            print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[1]]/A[C_min[1],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
        else: # C_max[1]==C_min[1]
            # Arbitrage elements in the same col
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[0] in location C_max[0]
            # then sell it for currency C_min[1] in location C_max[1]
            print('TRIANGULAR COLUMN')
            aer = np.real(1/C[C_min]*C[C_min[0], C_max[0]]*C[C_max[0],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
            print(f'BUY {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
            print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[1]]}({A[C_max[0],C_min[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4} = {operation:.5}')
    else:
        # Cuadrangular arbitrage
        # Arbitrage that involves four currencies and four locations
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[0] in location C_max[0]
        # then sell it for currency C_max[1] in location C_max[1]
        # then buy currency C_min[1] in location C_max[1]
        print('CUADRANGULAR')
        aer = np.real(1/C[C_min]*C[C_min[0],C_max[0]]*C[C_max]/C[C_min[1],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_max[1]]}({A[C_max[0],C_max[1]]}) in {locations[C_max[1]]}')
        print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]/A[C_min[1],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
        
    return aer

In [4]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [254]:
binance_uri = binance_url + binance_endpoints['exchange_info'][1]

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))

df_symbols = pd.DataFrame(JSON_object['symbols'])
df_symbols = df_symbols[df_symbols['status']=='TRADING']

In [310]:
assets = sorted(list(set(df_symbols['baseAsset'].unique()).union(set(df_symbols['quoteAsset'].unique()))))
symbols = sorted(list(df_symbols['symbol'].unique()))
len(assets), len(symbols)

(344, 1004)

In [603]:
%%time 

initial_currency = 'BTC'
selected_symbols = [s for s in symbols if initial_currency in s]

prices = []
for sym in selected_symbols:
    b_s_price = get_symbol_prices(sym)
    prices.append((sym, *b_s_price))
    
df_prices = pd.DataFrame(prices, columns=['symbol', 'buy', 'sell'])
df_prices.head()

CPU times: user 1.14 s, sys: 120 ms, total: 1.26 s
Wall time: 1min 42s


,symbol,buy,sell
0,1INCHBTC,0.000,0.000
1,AAVEBTC,0.006,0.006
2,ACMBTC,0.000,0.000
3,ADABTC,0.000,0.000
4,ADXBTC,0.000,0.000


In [604]:
df_prices['first_asset'] = np.NaN
for i in range(df_prices['symbol'].apply(len).max()):
    mask = df_prices['symbol'].str[0:i].isin(assets)
    df_prices.loc[mask, 'first_asset'] = df_prices[mask]['symbol'].str[0:i]

assert df_prices['first_asset'].isna().sum()==0, 'There are missing first_asset'


df_prices = df_prices.drop(index=df_prices[df_prices['first_asset'].apply(len)==df_prices['symbol'].apply(len)].index)
df_prices['second_asset'] = df_prices.apply(lambda x: x['symbol'][len(x['first_asset']):], axis=1)
    
assert (~df_prices['second_asset'].isin(assets)).sum()==0, 'There are missing second_asset'
assert (~(df_prices['first_asset']+df_prices['second_asset'] == df_prices['symbol'])).sum()==0, 'Symbol does not equal to asset1+asset2'

In [605]:
df_prices

,symbol,buy,sell,first_asset,second_asset
0,1INCHBTC,0.000,0.000,1INCH,BTC
1,AAVEBTC,0.006,0.006,AAVE,BTC
2,ACMBTC,0.000,0.000,ACM,BTC
3,ADABTC,0.000,0.000,ADA,BTC
4,ADXBTC,0.000,0.000,ADX,BTC
...,...,...,...,...,...
276,YOYOBTC,0.000,0.000,YOYO,BTC
277,ZECBTC,0.003,0.003,ZEC,BTC
278,ZENBTC,0.001,0.001,ZEN,BTC
279,ZILBTC,0.000,0.000,ZIL,BTC


---

In [610]:
final_assets = sorted(list(set(df_prices['first_asset']).union(set(df_prices['second_asset']))))

matrix_lite = pd.DataFrame(np.identity(len(final_assets)), index=final_assets, columns=final_assets)

for index, row in df_prices.iterrows():
    matrix_lite.loc[row['first_asset'], row['second_asset']] = row['buy']
    matrix_lite.loc[row['second_asset'], row['first_asset']] = 1/row['sell']
    
matrix_lite = matrix_lite.astype('float')
A = matrix_lite.to_numpy()

In [570]:
symbols_selection = ['ETH',
                     'LTC',
                     'BNB',
                     'NEO',
                     'BTC',
                     'IOTA',
                     'ETC',
                     'TRX',
                     'ADA',
                     'DOGE',
                     'USDT',
                     'EUR']

In [579]:
df_prices.set_index('symbol').loc['BTCEUR']

buy            48824.932
sell           48843.530
first_asset          BTC
second_asset         EUR
Name: BTCEUR, dtype: object

In [612]:
matrix_lite = pd.DataFrame(A, index=final_assets, columns=final_assets).replace(0,1)#.loc[symbols_selection,symbols_selection]

#matrix_lite = matrix_lite.loc[symbols_selection, symbols_selection]
matrix_lite

,1INCH,AAVE,ACM,ADA,ADX,AERGO,AGI,AION,AKRO,ALGO,...,XVG,XVS,YFI,YFII,YOYO,ZAR,ZEC,ZEN,ZIL,ZRX
1INCH,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
AAVE,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ACM,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ADA,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ADX,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAR,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ZEC,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ZEN,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
ZIL,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


---

In [560]:
def find_arbitrage():
    binance_uri = binance_url + binance_endpoints['price'][1] #+ '?symbol=BTCEUR'

    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    df_prices = pd.DataFrame(JSON_object)

    df_prices = df_prices[df_prices['symbol'].isin(symbols)]
    
    df_prices['first_asset'] = np.NaN
    for i in range(df_prices['symbol'].apply(len).max()):
        mask = df_prices['symbol'].str[0:i].isin(assets)
        df_prices.loc[mask, 'first_asset'] = df_prices[mask]['symbol'].str[0:i]

    assert df_prices['first_asset'].isna().sum()==0, 'There are missing first_asset'


    df_prices = df_prices.drop(index=df_prices[df_prices['first_asset'].apply(len)==df_prices['symbol'].apply(len)].index)
    df_prices['second_asset'] = df_prices.apply(lambda x: x['symbol'][len(x['first_asset']):], axis=1)

    assert (~df_prices['second_asset'].isin(assets)).sum()==0, 'There are missing second_asset'
    assert (~(df_prices['first_asset']+df_prices['second_asset'] == df_prices['symbol'])).sum()==0, 'Symbol does not equal to asset1+asset2'
    
    #########################################################
    matrix_lite = pd.DataFrame(np.identity(len(assets)), index=assets, columns=assets)

    for index, row in df_prices.iterrows():
        matrix_lite.loc[row['first_asset'], row['second_asset']] = row['price']

    matrix_lite = matrix_lite.astype('float')

    matrix_l = np.tril(matrix_lite.to_numpy().copy())
    upper_matrix = np.divide(1, matrix_l, out=np.zeros_like(matrix_l), where=matrix_l!=0).T

    matrix_u = np.triu(matrix_lite.to_numpy().copy())
    lower_matrix = np.divide(1, matrix_u, out=np.zeros_like(matrix_u), where=matrix_u!=0).T

    A = matrix_l + upper_matrix + matrix_u + lower_matrix - np.identity(matrix_l.shape[0])*3
    
    ####################################################
    
    currency_matrix = pd.DataFrame(A, index=assets, columns=assets).replace(0,1)
    currencies = locations = currency_matrix.columns.to_list()

    A = currency_matrix.to_numpy().copy()
    eigvals, eigvects = np.linalg.eig(A)

    idxmax = np.argmax(eigvals)
    valmax = eigvals[idxmax]
    vecmax = eigvects[:,idxmax]

    valapi = compute_API(valmax, len(A))


    if valapi>0:
        print(f'Arbitrage oportunity detected. API={valapi:.4}')
        aer = calculate_arbitrage(vecmax)
        if aer > 0.01:
            print(aer)
            return True
        else:
            return False
    else:
        print(f'Arbitrage oportunity no detected. API={valapi:.4}')
        return False

In [521]:
flag = False
while not flag:
    flag = find_arbitrage()

Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = 0.000%
BUY  BTCST/FIL(1.0) in FIL   
SELL FIL/BTCST(1.0) in BTCST 
1/1.0*1.0 = 1.0
Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = -0.000%
BUY  BTCST/FIL(1.0) in FIL   
SELL FIL/BTCST(1.0) in BTCST 
1/1.0*1.0 = 1.0
Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = 0.000%
BUY  BTCST/FIL(1.0) in FIL   
SELL FIL/BTCST(1.0) in BTCST 
1/1.0*1.0 = 1.0
Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = 0.000%
BUY  BTCST/FIL(1.0) in FIL   
SELL FIL/BTCST(1.0) in BTCST 
1/1.0*1.0 = 1.0
Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = 0.000%
BUY  BTCST/FIL(1.0) in FIL   
SELL FIL/BTCST(1.0) in BTCST 
1/1.0*1.0 = 1.0
Arbitrage oportunity detected. API=21.95
Arbitrage orders: DIRECT
FIL -> BTCST -> FIL : AER = 0.000%
BUY  BTCST/FI

KeyboardInterrupt: 

### try arbitrage algorithm

In [613]:
def compute_API(lambda_max, n):
    return np.abs(lambda_max-n)/(n-1)

In [616]:
calculate_arbitrage(vecmax)==0

Arbitrage orders: DIRECT
IDRT -> BTC -> IDRT : AER = 0.000%
BUY  BTC/IDRT(831363997.5440415) in IDRT  
SELL IDRT/BTC(1.1954638151672434e-09) in BTC   
1/8.314e+08*1.195e-09 = 1.0062


True

In [625]:
currency_matrix = matrix_lite
currencies = locations = currency_matrix.columns.to_list()

A = currency_matrix.to_numpy().copy()
eigvals, eigvects = np.linalg.eig(A)

idxmax = np.argmax(eigvals)
valmax = eigvals[idxmax]
vecmax = eigvects[:,idxmax]

valapi = compute_API(valmax, len(A))


if valapi>0:
    print(f'Arbitrage oportunity detected. API={valapi:.4}')
    calculate_arbitrage(vecmax)
else:
    print(f'Arbitrage oportunity no detected. API={valapi:.4}')

Arbitrage oportunity detected. API=6.757
Arbitrage orders: CUADRANGULAR
RENBTC -> BTC -> BTCDOWN -> BTC -> RENBTC : AER = 0.270%
BUY  BTC/RENBTC(1.0002012363209514) in RENBTC
SELL BTC/BTCDOWN(1.0) in BTCDOWN
SELL BTCDOWN/BTC(1.0) in BTC
BUY  RENBTC/BTC(0.9971046243645265) in BTC
1/1.0*1.0*1.0/0.9971 = 1.0027


In [623]:
B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
C = np.divide(A, B)

C_max = np.unravel_index(C.argmax(), C.shape)
C_min = np.unravel_index(C.argmin(), C.shape)

# Algorithm
# BUY = division
# SELL = multiply
print('Arbitrage orders: ', end="")
if C_max[0]==C_min[1] and C_max[1]==C_min[0]:
    # Direct arbitrage
    # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
    # and sell it for currency C_max[0] in location C_max[1]
    print('DIRECT')
    aer = 1/np.real(C[C_min]*C[C_max])-1

    print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
    print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]:6}')
    print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[0]]}({A[C_max[0],C_min[0]]}) in {locations[C_max[1]]:6}')

    operation = 1/(A[C_min]*A[C_max])
    print(f'1/{A[C_min]:.4}*{A[C_max]:.4} = {operation:.5}')

elif C_max[0]==C_min[0] or C_max[1]==C_min[1]:
    # Triangular arbitrage
    if C_max[0]==C_min[0]:
        # Arbitrage elements in the same row
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[1] in location C_max[1]
        # then buy currency C_max[1] in location C_max[1]
        print('TRIANGULAR ROW')
        aer = np.real(1/C[C_min]*C[C_min[0],C_max[1]]/C[C_min[1],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[1]]}({A[C_min[0],C_max[1]]}) in {locations[C_max[1]]}')
        print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[1]]/A[C_min[1],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
    else: # C_max[1]==C_min[1]
        # Arbitrage elements in the same col
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[0] in location C_max[0]
        # then sell it for currency C_min[1] in location C_max[1]
        print('TRIANGULAR COLUMN')
        aer = np.real(1/C[C_min]*C[C_min[0], C_max[0]]*C[C_max[0],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[1]]}({A[C_max[0],C_min[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4} = {operation:.5}')
else:
    # Cuadrangular arbitrage
    # Arbitrage that involves four currencies and four locations
    # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
    # then sell it for currency C_max[0] in location C_max[0]
    # then sell it for currency C_max[1] in location C_max[1]
    # then buy currency C_min[1] in location C_max[1]
    print('CUADRANGULAR')
    aer = np.real(1/C[C_min]*C[C_min[0],C_max[0]]*C[C_max]/C[C_min[1],C_max[1]]-1)

    print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
    print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
    print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
    print(f'SELL {currencies[C_max[0]]}/{currencies[C_max[1]]}({A[C_max[0],C_max[1]]}) in {locations[C_max[1]]}')
    print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
    operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]/A[C_min[1],C_max[1]]
    print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')

Arbitrage orders: CUADRANGULAR
RENBTC -> BTC -> BTCDOWN -> BTC -> RENBTC : AER = 0.270%
BUY  BTC/RENBTC(1.0002012363209514) in RENBTC
SELL BTC/BTCDOWN(1.0) in BTCDOWN
SELL BTCDOWN/BTC(1.0) in BTC
BUY  RENBTC/BTC(0.9971046243645265) in BTC
1/1.0*1.0*1.0/0.9971 = 1.0027


In [586]:
pd.DataFrame(B, index=symbols_selection, columns=symbols_selection)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


,ETH,LTC,BNB,NEO,BTC,IOTA,ETC,TRX,ADA,DOGE,USDT,EUR
ETH,1.000,21.326,6.644,117.066,0.003,678.751,309.512,803.170,697.842,56.823,191.847,166.682
LTC,0.047,1.000,0.312,5.489,0.000,31.828,14.513,37.662,32.723,2.665,8.996,7.816
BNB,0.151,3.210,1.000,17.620,0.001,102.159,46.585,120.886,105.033,8.553,28.875,25.088
NEO,0.009,0.182,0.057,1.000,0.000,5.798,2.644,6.861,5.961,0.485,1.639,1.424
BTC,292.530,6238.477,1943.580,34245.394,1.000,198555.159,90541.561,234951.309,204139.842,16622.524,56120.920,48759.606
IOTA,0.001,0.031,0.010,0.172,0.000,1.000,0.456,1.183,1.028,0.084,0.283,0.246
ETC,0.003,0.069,0.021,0.378,0.000,2.193,1.000,2.595,2.255,0.184,0.620,0.539
TRX,0.001,0.027,0.008,0.146,0.000,0.845,0.385,1.000,0.869,0.071,0.239,0.208
ADA,0.001,0.031,0.010,0.168,0.000,0.973,0.444,1.151,1.000,0.081,0.275,0.239
DOGE,0.018,0.375,0.117,2.060,0.000,11.945,5.447,14.135,12.281,1.000,3.376,2.933


In [587]:
pd.DataFrame(C, index=symbols_selection, columns=symbols_selection)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


,ETH,LTC,BNB,NEO,BTC,IOTA,ETC,TRX,ADA,DOGE,USDT,EUR
ETH,1.000,0.047,0.151,0.009,292.530,0.001,0.003,0.001,0.001,0.018,0.005,0.006
LTC,21.326,1.000,3.210,0.182,6238.477,0.031,0.069,0.027,0.031,0.375,0.111,0.128
BNB,6.644,0.312,1.000,0.057,1943.580,0.010,0.021,0.008,0.010,0.117,0.035,0.040
NEO,117.066,5.489,17.620,1.000,34245.394,0.172,0.378,0.146,0.168,2.060,0.610,0.702
BTC,0.003,0.000,0.001,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
IOTA,678.751,31.828,102.159,5.798,198555.159,1.000,2.193,0.845,0.973,11.945,3.538,4.072
ETC,309.512,14.513,46.585,2.644,90541.561,0.456,1.000,0.385,0.444,5.447,1.613,1.857
TRX,803.170,37.662,120.886,6.861,234951.309,1.183,2.595,1.000,1.151,14.135,4.187,4.819
ADA,697.842,32.723,105.033,5.961,204139.842,1.028,2.255,0.869,1.000,12.281,3.637,4.187
DOGE,56.823,2.665,8.553,0.485,16622.524,0.084,0.184,0.071,0.081,1.000,0.296,0.341


In [588]:
pd.Series(vecmax, index=symbols_selection).sort_values().tail(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


IOTA   0.000
ETC    0.000
USDT   0.000
EUR    0.000
NEO    0.000
DOGE   0.000
LTC    0.000
BNB    0.001
ETH    0.003
BTC    1.000
dtype: complex128